
## Semantic Kernel with MCP

A comprehensive list of all MCP servers can be found [here](https://github.com/modelcontextprotocol/servers?tab=readme-ov-file). Ensure that your arguments (`args`) and environment variables (`env`) conform to the MCP input requirements. Additionally, create the necessary API keys if required by the MCP server.

### Example: Using Google Maps MCP  ],
      "args": [
        "-y",
        "@modelcontextprotocol/server-google-maps"
      ],
      "env": {
        "GOOGLE_MAPS_API_KEY": "<YOUR_API_KEY>"
      }

In [ ]:
import asyncio
import nest_asyncio
import os
from dotenv import load_dotenv
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin

load_dotenv()
nest_asyncio.apply()

async def main():
    async with MCPStdioPlugin(
        name="Github",
        description="Github Plugin",
        command="npx",
        args=["-y", "@modelcontextprotocol/server-github"],
    ) as github_plugin:
        agent = ChatCompletionAgent(
            service=AzureChatCompletion(),
            name="IssueAgent",
            instructions="Answer questions about gitHub repos, issues etc.",
            plugins=[github_plugin],
        )

        thread: ChatHistoryAgentThread = None

        print("💬 Start chatting with the agent. Type 'exit' to quit.\n")

        while True:
            try:
                user_input = input("User:> ")
            except (KeyboardInterrupt, EOFError):
                print("\n\n👋 Exiting chat...")
                break

            if user_input.lower().strip() == "exit":
                print("\n👋 Exiting chat...")
                break

            response = await agent.get_response(messages=user_input, thread=thread)
            print(f"{response.name}: {response}")
            thread = response.thread

        # Cleanup: delete the thread
        await thread.delete() if thread else None


In [ ]:
asyncio.run(main())